In [ ]:
!pip install pyomo==6.6.2
!apt-get install -y -qq glpk-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.8 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package 

In [ ]:
import pandas as pd
import numpy as np
import sys
import matplotlib as plt
from pyomo.environ import *

In [ ]:
# Parameters
capacity = {'coal': 300, 'natural_gas': 500, 'solar': 200, 'wind': 300, 'grid': 1000}  # MW
cost_per_mwh = {'coal': 60, 'natural_gas': 55, 'solar': 25, 'wind': 20, 'grid': 70}  # $/MWh
emissions_rate = {'coal': 1.0, 'natural_gas': 0.5, 'solar': 0, 'wind': 0, 'grid': 0.45}  # tons/MWh
cap_factor = {'coal': 0.85, 'natural_gas': 0.8, 'solar': 0.3, 'wind': 0.35, 'grid': 1}  # %

# Budget and demand limits
demand = 1000000  # MWh (Annual demand)
budget = 50000000  # $ (Budget)
carbon_budget = 5000000  # $ for carbon costs (Carbon budget)

# Create a Pyomo model
model = ConcreteModel()

# Define sets
model.sources = Set(initialize=capacity.keys())

# Define variables
model.generation = Var(model.sources, within=NonNegativeReals)
model.carbon_credits = Var(within=NonNegativeReals)

# Constraints

def capacity_constraint_rule(model, s):
    return model.generation[s] <= capacity[s] * cap_factor[s] * 8760
model.capacity_constraints = Constraint(model.sources, rule=capacity_constraint_rule)

def demand_constraint_rule(model):
    return sum(model.generation[s] for s in model.sources) == demand
model.demand_constraint = Constraint(rule=demand_constraint_rule)

def budget_constraint_rule(model):
    return sum(model.generation[s] * cost_per_mwh[s] for s in model.sources) <= budget
model.budget_constraint = Constraint(rule=budget_constraint_rule)

def carbon_constraint_rule(model):
    return sum(model.generation[s] * emissions_rate[s] for s in model.sources) * 20 - model.carbon_credits * 20 <= carbon_budget
model.carbon_constraint = Constraint(rule=carbon_constraint_rule)

def grid_constraint_rule(model):
    return model.generation['grid'] >= 0.1 * demand
model.grid_constraint = Constraint(rule=grid_constraint_rule)

def natural_gas_constraint_rule(model):
    return model.generation['natural_gas'] >= 1000
model.natural_gas_constraint = Constraint(rule=natural_gas_constraint_rule)

# Objective function (minimize emissions)
model.objective = Objective(expr=sum(model.generation[s] * emissions_rate[s] for s in model.sources), sense=minimize)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

In [ ]:
# Display results
if results.solver.status == SolverStatus.ok:
    print("Generation Results:")
    total_operating_costs = 0  # Initialize total operating costs
    total_emissions = 0       # Initialize total emissions

    for s in model.sources:
        generated_mwh = value(model.generation[s])
        emitted_co2 = generated_mwh * emissions_rate[s]
        cost_spent = generated_mwh * cost_per_mwh[s]

        total_operating_costs += cost_spent  # Accumulate operating costs
        total_emissions += emitted_co2       # Accumulate emissions

        print(f"From {s.capitalize()}, generate {generated_mwh:.2f} MWh. "
              f"CO2 emitted: {emitted_co2:.2f} tons. Cost: ${cost_spent:.2f}.")

    # Calculate carbon credits needed
    carbon_credits_needed = total_emissions  # Assume all emissions are offset

    # Calculate total cost for carbon credits
    carbon_credit_cost = carbon_credits_needed * 20  # $20 per ton

    # Check if carbon credit cost is within budget
    if carbon_credit_cost <= carbon_budget:
        print(f"\nTotal CO2 Emitted: {total_emissions:.2f} tons.")
        print(f"Carbon credits purchased: {carbon_credits_needed:.2f} tons.")
        print(f"Total cost for carbon credits: ${carbon_credit_cost:.2f}.")

        # Calculate remaining budget after buying carbon credits
        remaining_energy_budget = budget - total_operating_costs
        remaining_carbon_budget = carbon_budget - carbon_credit_cost
        print(f"Total Operating Costs: ${total_operating_costs:.2f}")
        print(f"Remaining Energy Budget: ${remaining_energy_budget:.2f}")
        print(f"Remaining Carbon Budget: ${remaining_carbon_budget:.2f}")
    else:
        print(f"Not enough budget for carbon offsets. Needed: ${carbon_credit_cost:.2f}, Available: ${carbon_budget:.2f}.")


else:
    print("Optimization failed:", results.solver.termination_condition)

Generation Results:
From Coal, generate 0.00 MWh. CO2 emitted: 0.00 tons. Cost: $0.00.
From Natural_gas, generate 1000.00 MWh. CO2 emitted: 500.00 tons. Cost: $55000.00.
From Solar, generate 0.00 MWh. CO2 emitted: 0.00 tons. Cost: $0.00.
From Wind, generate 899000.00 MWh. CO2 emitted: 0.00 tons. Cost: $17980000.00.
From Grid, generate 100000.00 MWh. CO2 emitted: 45000.00 tons. Cost: $7000000.00.

Total CO2 Emitted: 45500.00 tons.
Carbon credits purchased: 45500.00 tons.
Total cost for carbon credits: $910000.00.
Total Operating Costs: $25035000.00
Remaining Energy Budget: $24965000.00
Remaining Carbon Budget: $4090000.00
